In [19]:
import pytesseract
from PIL import Image
import re
import pandas as pd


#### Load image

In [20]:

image_path = r"C:\Users\abdel\OneDrive\Desktop\New folder (3)\ocr\1244.jpeg"


#### Open image using PIL

In [21]:

image = Image.open(image_path)


#### Perform OCR on the image

In [ ]:
extracted_text = pytesseract.image_to_string(image)


#### Split extracted text into lines

In [ ]:

lines = extracted_text.split("\n")


In [ ]:
products = []
prix_unitaire = []
prix_total = []

#### Track sections manually

In [25]:

collecting_products = True
collecting_unitaire = False
collecting_total = False

for line in lines:
    line = line.strip()
    if not line:
        continue
    
    # Skip store header information
    if line.startswith("MG PROXI ARIANA ESSOGHRA") or line.startswith("AVENUE FETHI ZQUHEIR"):
        continue
    
    # Remove unwanted symbols
    line = re.sub(r'[^0-9a-zA-Z., ]', '', line)
    
    # Detect transition to unit prices section
    if collecting_products and all(c.isdigit() or c in ",." for c in line.replace(" ", "")):
        collecting_products = False
        collecting_unitaire = True
        prix_unitaire.append(line)  # Capture the first price immediately
        continue  
    
    # Detect transition to total prices section
    elif collecting_unitaire and len(prix_unitaire) >= len(products):
        collecting_unitaire = False
        collecting_total = True
    
    # Collect data based on section
    if collecting_products:
        products.append(line)
    elif collecting_unitaire:
        prix_unitaire.append(line)
    elif collecting_total:
        prix_total.append(line)


#### Ensure all lists have the same length

In [26]:


max_length = max(len(products), len(prix_unitaire), len(prix_total))
products += ["" for _ in range(max_length - len(products))]
prix_unitaire += ["" for _ in range(max_length - len(prix_unitaire))]
prix_total += ["" for _ in range(max_length - len(prix_total))]



#### Create DataFrame

In [ ]:

receipt_df = pd.DataFrame({
    "Product": products,
    "Prix Unitaire": prix_unitaire,
    "Prix Total": prix_total
})

print(receipt_df)


           Product Prix Unitaire Prix Total
0    1 SAUCE PIZZA         2.200      2.200
1   1 CREME FRATCH         3.120      3.120
2   1 PATE PAPILLO         1.120      1,120
3     MARGARINE 20         2.990      2.990
4     MORCEAUX THO         3.490      3.490
5   1 CREAME A TAR         7,900      7.500
6     TIMBRE 100 M         0,100      0.100
7   1 Y.A.B VANILL         0.960      0.960
8    1 BOIS.LACTEE         0.980     0.980 
9   2 FROMAGE GJUT         0.890      1.780
10                                       at
11   1 SAUCE PIZZA         2.200      2.200
12  1 CREME FRATCH         3.120      3.120
13  1 PATE PAPILLO         1.120      1,120
14    MARGARINE 20         2.990      2.990
15    MORCEAUX THO         3.490      3.490
16  1 CREAME A TAR         7,900      7.500
17    TIMBRE 100 M         0,100      0.100
18  1 Y.A.B VANILL         0.960      0.960
19   1 BOIS.LACTEE         0.980     0.980 
20  2 FROMAGE GJUT         0.890      1.780
21   1 SAUCE PIZZA         2.200